In [1]:
from manim import *
import random 

/home/codespace/.python/current/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


Manim Community v0.18.0.post0

In [37]:
%%manim -ql --fps 30 -v WARNING ImpredicativityAnimation

class ImpredicativityAnimation(Scene):
    def construct(self):
        self.camera.background_color = GREY

        # Create the spheres
        transformer = Circle(radius=1, color=BLUE, fill_color=BLUE, fill_opacity=1).shift(LEFT*2)
        transformer.set_z_index(1)
        #ejector = Circle(radius=1, color=WHITE).shift(RIGHT*2)
        ejector = Square(side_length=2, color=BLACK, fill_color=BLACK, fill_opacity=1).shift(RIGHT*2)
        ejector.set_z_index(1)  # Ensure the square is always in front
        #transformer_label = Text("Transformer", font_size=24).next_to(transformer, DOWN)
        #ejector_label = Text("Ejector", font_size=24).next_to(ejector, DOWN)
        #self.add(transformer, ejector, transformer_label, ejector_label)
        self.add(transformer, ejector)

        for _ in range(4):  # Loop the animation sequence twice
            # Initial ball properties
            ball = Circle(radius=0.2, color=WHITE, fill_color=WHITE, fill_opacity=1).move_to(LEFT * 8)

            # Animation: ball moves to transformer and changes color
            self.play(ball.animate.move_to(transformer.get_center()), run_time=2)
            self.play(ball.animate.set_color(transformer.color))
            
            # Animation: ball moves to ejector
            self.play(ball.animate.move_to(ejector.get_center()), run_time=2)

            new_color = random_bright_color(transformer.color)  # New color from a helper function
            square = Square(side_length=0.2, color=new_color, fill_color=new_color, fill_opacity=1).move_to(ejector.get_center())

            # Define a curved path to the transformer
            path = ArcBetweenPoints(start=ejector.get_center(), 
                                    end=transformer.get_center() + UP * transformer.radius + DOWN * ball.radius, 
                                    angle=TAU/2)

            # Combined animation sequence
            self.play(
                MoveAlongPath(square, path),
                run_time=4,  # Adjust total duration as needed
            )

            # Ball arrives and changes the color of the transformer and fades out simultaneously
            self.play(
                transformer.animate.set_color(new_color),
                FadeOut(square)
            )
            
            self.wait(2)  # Pause before next loop iteration

        self.play(*[FadeOut(mob) for mob in self.mobjects])
        self.wait(2) 

# Helper function to get random bright colors
import random

def random_bright_color(current_color):
    colors = [RED, YELLOW, PURPLE]
    colors = [color for color in colors if color != current_color]
    return random.choice(colors)



In [ ]:
%%manim -ql --fps 30 -v WARNING LayeredGraphProjection

import networkx as nx
import numpy as np

class LayeredGraphProjection(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=70 * DEGREES, theta=135 * DEGREES)

        # Create a graph with layers
        G = nx.Graph()
        layers = {1: 0, 2: 1, 3: 1, 4: 2, 5: 2, 6: 0}
        edges = [(1, 2), (1, 3), (3, 4), (2, 4), (2, 5), (4, 5), (6, 5)]
        G.add_edges_from(edges)

        # Define colors for each layer
        layer_colors = {
            0: BLUE,
            1: GREEN,
            2: RED
        }

        # 2D layout for base projection
        pos = nx.spring_layout(G, dim=2)
        z_scale = 1  # Scale for the z-axis to visually separate layers more clearly
        pos_3d = {node: np.array([*pos[node], layers[node] * z_scale]) for node in G.nodes}

        # Create 3D nodes and edges
        nodes_3d = [Sphere(radius=0.1, color=layer_colors[layers[node]]).move_to(pos_3d[node]) for node in G.nodes]
        edges_3d = [Line3D(start=pos_3d[edge[0]], end=pos_3d[edge[1]], color=WHITE) for edge in G.edges]

        # Calculate minimum Z for the projection plane to sit below the lowest node
        min_z = min(layers.values()) * z_scale - 1  # One unit below the lowest layer

        # Maximum layer for scaling purposes
        max_layer = max(layers.values())

        # Create 2D nodes and edges with perspective scaling
        nodes_2d = [Dot(point=[*pos[node], min_z], color=layer_colors[layers[node]], radius=0.1 * (1 + (layers[node] / (max_layer + 1))))
                    for node in G.nodes]
        edges_2d = [Line(start=[*pos[edge[0]], min_z], end=[*pos[edge[1]], min_z], color=LIGHT_GRAY) for edge in G.edges]

        # Projection lines connecting 3D and 2D nodes
        projection_lines = [DashedLine(start=pos_3d[node], end=[*pos[node], min_z], color=GRAY) for node in G.nodes]

        # Create a transparent square surface for the 2D projection plane
        plane_size = 3.5  # Size of each plane, adjust as necessary
        projection_plane = Rectangle(width=plane_size, height=plane_size, fill_color=BLUE, fill_opacity=0.5, stroke_width=0)
        projection_plane.shift(OUT * min_z)  # Positioned one step below the lowest node

        # Create planes for each layer
        unique_layers = set(layers.values())
        layer_planes = []
        for layer in unique_layers:
            plane = Rectangle(width=plane_size, height=plane_size, fill_color=layer_colors[layer], fill_opacity=0.2, stroke_width=0)
            plane.shift(OUT * layer * z_scale)  # Offset each plane to match its respective layer height
            layer_planes.append(plane)

        # Group all elements including the planes
        graph_group = Group(*projection_plane, *layer_planes, *nodes_3d, *edges_3d, *nodes_2d, *edges_2d, *projection_lines)

        # Add the entire graph and projection in one step
        #self.play(FadeIn(graph_group), run_time=1)
        self.add(graph_group)

        # Camera swing around the scene
        self.move_camera(phi=80 * DEGREES, theta=-90 * DEGREES, run_time=1)

        #self.wait(1)  # Short wait time to view the complete graph and projection after the camera swing

# Render with a lower frame rate if necessary: manim -pql --fps 30 script.py


In [ ]:
%%manim render --help

pass

Usage: manim render [OPTIONS] FILE [SCENE_NAMES]...

  Render SCENE(S) from the input FILE.

  FILE is the file path of the script or a config file.

  SCENES is an optional list of scenes in the file.

Global options:
  -c, --config_file TEXT         Specify the configuration file to use for
                                 render settings.
  --custom_folders               Use the folders defined in the [custom_folders]
                                 section of the config file to define the output
                                 folder structure.
  --disable_caching              Disable the use of the cache (still generates
                                 cache files).
  --flush_cache                  Remove cached partial movie files.
  --tex_template TEXT            Specify a custom TeX template file.
  -v, --verbosity [DEBUG|INFO|WARNING|ERROR|CRITICAL]
                                 Verbosity of CLI output. Changes ffmpeg log
                                 level unless 5+.